In [1]:
print("Jai Shree Ram")

Jai Shree Ram


In [ ]:
"""
C86 Client 360 - RBC
Author: Maharaj Aryan Kumar
Date: 09/11/2025
Supervisor: AP
"""


Screenshot 1
Detects DEV/PROD, sets base folders, creates today’s log/listing files, and redirects SAS logging/printing to those files.

In [ ]:
from pathlib import Path
import os
import socket
from datetime import datetime
import logging
import pandas as pd
from datetime import timedelta

# --- Base directory: where the script is located (or current working directory in notebooks) ---
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()   # fallback for Jupyter/interactive

# --- Helper function ---
def ensure_dir(path: Path):
    """Create folder if it does not exist."""
    path.mkdir(parents=True, exist_ok=True)
    return path

# 1) Detect environment: DEV or PROD
user = (os.getenv("USER") or os.getenv("USERNAME") or "").strip()
host = socket.gethostname().lower()
env = "PROD" if (user[:1].upper() == "U" and host != "usasst11") else "DEV"

# 2) Base path inside project folder
regpath = BASE_DIR / ("REG" if env == "PROD" else "REG_DEV")

# 3) Output folders
logpath = ensure_dir(regpath / "C86" / "log" / "product_appropriateness" / "client360")
outpath = ensure_dir(regpath / "output" / "product_appropriateness" / "client360")

# 4) Daily filenames (yyyymmdd)
ymd = datetime.now().strftime("%Y%m%d")
logfile = logpath / f"C86_pa_client360_{ymd}.log"
lstfile = logpath / f"C86_pa_client360_{ymd}.lst"

# 5) Logging setup
logging.basicConfig(
    filename=str(logfile),
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)
logging.info("Starting run...")
logging.info("User=%s Host=%s Env=%s", user, host, env)
logging.info("regpath=%s", str(regpath))
logging.info("logpath=%s", str(logpath))
logging.info("outpath=%s", str(outpath))

# Also mimic SAS listing output
with open(lstfile, "a", encoding="utf-8") as f:
    f.write(f">>> Starting Running Time - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Environment: {env}\n")
    f.write(f"User: {user}  Host: {host}\n")
    f.write(f"regpath: {regpath}\n\n")

# 6) Capture run configuration
run_config = pd.DataFrame([{
    "user": user,
    "host": host,
    "env": env,
    "regpath": str(regpath),
    "logpath": str(logpath),
    "outpath": str(outpath),
    "logfile": str(logfile),
    "lstfile": str(lstfile),
    "run_datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
}])

run_config

,user,host,env,regpath,logpath,outpath,logfile,lstfile,run_datetime
0,maharaj,aryans-macbook-pro.local,DEV,/Users/maharaj/Documents/C86Client360_09112025...,/Users/maharaj/Documents/C86Client360_09112025...,/Users/maharaj/Documents/C86Client360_09112025...,/Users/maharaj/Documents/C86Client360_09112025...,/Users/maharaj/Documents/C86Client360_09112025...,2025-09-10 09:47:16


Screenshot 2
Initial-run switch + date window

Checks: 

client360_autocomplete.sas7bdat already exists in the output folder.

If it exists → marks this as not an initial run, and renames that file as a timestamped backup.

If it doesn’t → treats this as the initial run.

Sets up simple dates: today, a week window (start/end), and runday.

For the very first run, it uses fixed launch dates; for later runs, it uses the current week.

In [12]:
# 1) Check for existing dataset to decide initial run
autocomplete_file = outpath / "pa_client360_autocomplete.sas7bdat"
if autocomplete_file.exists():
    ini_run = "N"
    backup_name = f"pa_client360_autocomplete_backup_{datetime.now().strftime('%Y%m%d%H%M%S')}.sas7bdat"
    try:
        autocomplete_file.rename(outpath / backup_name)
    except Exception as e:
        logging.warning("Could not rename existing autocomplete file: %s", e)
else:
    ini_run = "Y"

# 2) Today's date and launch reference dates (match SAS literals)
from datetime import date, timedelta

tday = date.today()
launch_dt = date(2023, 5, 7)         # '07MAY2023'd
launch_dt_min14 = date(2023, 4, 23)  # '23APR2023'd

# 3) Week window (Monday..Sunday). Adjust if needed.
week_start = tday - timedelta(days=tday.weekday())
week_end = week_start + timedelta(days=6)

# 4) Choose window based on initial run
if ini_run == "Y":
    wk_start = launch_dt
    wk_start_min14 = launch_dt_min14
    wk_end = week_end
else:
    wk_start = week_start
    wk_start_min14 = week_start - timedelta(days=14)
    wk_end = week_end

# 5) Handy strings like SAS yyyymmdd
runday = tday.strftime("%Y%m%d")
tday_str = tday.strftime("%Y%m%d")
wk_start_str = wk_start.strftime("%Y-%m-%d")
wk_start_min14_str = wk_start_min14.strftime("%Y-%m-%d")
wk_end_str = wk_end.strftime("%Y-%m-%d")

# 6) Create daily output folder (mimics libname dataout "&outpath/&runday")
runday_folder = outpath / runday
runday_folder.mkdir(parents=True, exist_ok=True)

# 7) Show results in a DataFrame
run_dates = pd.DataFrame([{
    "ini_run": ini_run,
    "wk_start": wk_start_str,
    "wk_start_min14": wk_start_min14_str,
    "wk_end": wk_end_str,
    "runday": runday,
    "tday": tday_str,
    "outpath": str(outpath),
    "runday_folder": str(runday_folder),
}])

print("\nRun dates:\n", run_dates.to_string(index=False))


Run dates:
 ini_run   wk_start wk_start_min14     wk_end   runday     tday                                                                                         outpath                                                                                            runday_folder
      Y 2023-05-07     2023-04-23 2025-09-14 20250910 20250910 /Users/maharaj/Documents/C86Client360_09112025/REG_DEV/output/product_appropriateness/client360 /Users/maharaj/Documents/C86Client360_09112025/REG_DEV/output/product_appropriateness/client360/20250910


=============================================================================

Screenshot 3 — Pull data from Tracking (Teradata) + simple aggregations

What it does: Connects to Teradata, pulls recent "Advice Tool" events

(EVNT_DT > wk_start - 90 days), then builds two beginner-friendly DataFrames:

    1) tracking_tool_use_distinct: distinct (OPPOR_ID, ADVC_TOOL_NM)

    2) tracking_count_tool_use_pre2: per OPPOR_ID, count of unique tools used

Results are also saved as CSVs into the runday folder.

=============================================================================

In [14]:
import json
import logging
from pathlib import Path
from datetime import timedelta

# Helper: get a Teradata connection using a small JSON file.
def get_teradata_conn(config_path: str = "TeradataConnection_T.json"):
    try:
        import teradatasql  # pip install teradatasql
    except Exception:
        logging.warning("teradatasql not available; returning None. Install with: pip install teradatasql")
        return None

    cfg = Path(config_path)
    if not cfg.exists():
        logging.warning("Config file %s not found. Skipping DB fetch.", config_path)
        return None

    with open(cfg) as f:
        creds = json.load(f)

    try:
        conn = teradatasql.connect(
            host=creds["url"],
            user=creds["user"],
            password=creds["password"],
            logmech="LDAP",
        )
        return conn
    except Exception as e:
        logging.error("Teradata connection failed: %s", e)
        return None


# --- Compute the 90-day lookback from wk_start (defined earlier) ---
start_90 = wk_start - timedelta(days=90)
start_literal = start_90.strftime("%Y-%m-%d")  # DATE 'YYYY-MM-DD' in SQL

# --- Columns we actually need ---
cols = ["OPPOR_ID", "ADVC_TOOL_NM"]

# --- Default empty frame so code runs even without DB access ---
tracking_all = pd.DataFrame(columns=cols)

conn = get_teradata_conn()
if conn is not None:
    try:
        sql = f"""
            SELECT OPPOR_ID,
                   ADVC_TOOL_NM
            FROM DDW01.EVNT_PROD_TRACK_LOG
            WHERE advr_selt_typ = 'Advice Tool'
              AND EVNT_DT > DATE '{start_literal}'
        """
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        colnames = [d[0] for d in cur.description]
        tracking_all = pd.DataFrame(rows, columns=colnames).reindex(cols, axis=1)
        cur.close()
        conn.close()
    except Exception as e:
        logging.error("Query failed: %s", e)

# --- Clean & standardize ---
if not tracking_all.empty:
    tracking_all = tracking_all.dropna(subset=["OPPOR_ID", "ADVC_TOOL_NM"]).copy()
    tracking_all["ADVC_TOOL_NM"] = tracking_all["ADVC_TOOL_NM"].astype(str).str.upper()

# --- 1) Distinct (OPPOR_ID, ADVC_TOOL_NM) ---
tracking_tool_use_distinct = tracking_all.drop_duplicates(subset=["OPPOR_ID", "ADVC_TOOL_NM"])

# --- 2) Count of unique tools used per OPPOR_ID ---
tracking_count_tool_use_pre2 = (
    tracking_all
    .groupby("OPPOR_ID", dropna=False)["ADVC_TOOL_NM"]
    .nunique()
    .reset_index(name="count_unique_tool_used")
    .sort_values("count_unique_tool_used", ascending=False)
)

# --- Save simple CSV outputs (mimics SAS datasets) ---
tracking_all.to_csv(runday_folder / "tracking_all.csv", index=False)
tracking_tool_use_distinct.to_csv(runday_folder / "tracking_tool_use_distinct.csv", index=False)
tracking_count_tool_use_pre2.to_csv(runday_folder / "tracking_count_tool_use_pre2.csv", index=False)

print("\nRows pulled:", len(tracking_all))
print("Distinct pairs:", len(tracking_tool_use_distinct))
print("OPPOR_ID tool-count rows:", len(tracking_count_tool_use_pre2))


Rows pulled: 0
Distinct pairs: 0
OPPOR_ID tool-count rows: 0


Screenshot 4 — “Tool Used” flag from aggregate

What it does (in short):
Takes the aggregated table tracking_count_tool_use_pre2 (unique tool count per OPPOR_ID) and builds a light flag table:

OPPOR_ID

tool_used = 'Tool Used' if count_unique_tool_used > 0, otherwise blank.

It also saves tracking_tool_use.csv to the runday folder.

In [18]:
import numpy as np

# Guard: make sure upstream DataFrame exists
tracking_count_tool_use_pre2 = locals().get(
    "tracking_count_tool_use_pre2",
    pd.DataFrame(columns=["OPPOR_ID", "count_unique_tool_used"])  # empty fallback
)

tracking_tool_use = (
    tracking_count_tool_use_pre2[["OPPOR_ID", "count_unique_tool_used"]].copy()
)

# Mark if tool was used (mimics SAS logic)
tracking_tool_use["tool_used"] = np.where(
    tracking_tool_use["count_unique_tool_used"] > 0,
    "Tool Used",
    None
)

# Keep SAS-like column order
tracking_tool_use = tracking_tool_use[["OPPOR_ID", "tool_used"]]

# Save CSV
tracking_tool_use.to_csv(runday_folder / "tracking_tool_use.csv", index=False)

print("tracking_tool_use rows:", len(tracking_tool_use))

tracking_tool_use rows: 0


Screenshot 5 — “c360_short + c360_detail_pre (employee attributes by snapshot date)”

What it does (in short):

Builds a snapshot view c360_short from ddw01.evnt_prod_oppor with:

evnt_id, emp_id = CAST(rbc_oppor_own_id AS INTEGER), snap_dt = evnt_dt

Filters rows between wk_start and wk_end and requires ids/dates to be not null.

Creates c360_detail_pre by joining that snapshot to HR tables (ddw01.emp, ddw01.empl_reltn) where the snapshot date falls between captr_dt and chg_dt (SCD-style “valid on date” join).

Saves both DataFrames as CSVs in the runday folder.

In [20]:
wk_start_lit = wk_start.strftime("%Y-%m-%d")
wk_end_lit = wk_end.strftime("%Y-%m-%d")

c360_short = pd.DataFrame(columns=["evnt_id", "emp_id", "snap_dt"])  # default empty
c360_detail_pre = pd.DataFrame()  # default empty

conn = get_teradata_conn()
if conn is not None:
    try:
        # 1) Pull c360_short
        sql_short = f"""
            SELECT evnt_id,
                   CAST(rbc_oppor_own_id AS INTEGER) AS emp_id,
                   evnt_dt AS snap_dt
            FROM ddw01.evnt_prod_oppor
            WHERE rbc_oppor_own_id IS NOT NULL
              AND evnt_dt IS NOT NULL
              AND evnt_id IS NOT NULL
              AND evnt_dt BETWEEN DATE '{wk_start_lit}' AND DATE '{wk_end_lit}'
        """
        with conn.cursor() as cur:
            cur.execute(sql_short)
            rows = cur.fetchall()
            cols = [d[0] for d in cur.description]
            c360_short = pd.DataFrame(rows, columns=cols)

        # 2) Pull detailed attributes
        sql_detail = f"""
            WITH c360_short AS (
                SELECT evnt_id,
                       CAST(rbc_oppor_own_id AS INTEGER) AS emp_id,
                       evnt_dt AS snap_dt
                FROM ddw01.evnt_prod_oppor
                WHERE rbc_oppor_own_id IS NOT NULL
                  AND evnt_dt IS NOT NULL
                  AND evnt_id IS NOT NULL
                  AND evnt_dt BETWEEN DATE '{wk_start_lit}' AND DATE '{wk_end_lit}'
            )
            SELECT
                c360.evnt_id,
                c360.evnt_dt,
                c360.rbc_oppor_own_id,
                emp.org_unt_no,
                emp.hr_posn_titl_en,
                emp.posn_strt_dt,
                emp.posn_end_dt,
                emp.occpt_job_cd
            FROM ddw01.evnt_prod_oppor AS c360
            LEFT JOIN (
                SELECT c3.evnt_id,
                       e1.org_unt_no,
                       e1.hr_posn_titl_en,
                       e2.posn_strt_dt,
                       e2.posn_end_dt,
                       e1.occpt_job_cd
                FROM c360_short AS c3
                INNER JOIN ddw01.emp AS e1
                  ON e1.emp_id = c3.emp_id
                 AND c3.snap_dt >= e1.captr_dt
                 AND c3.snap_dt <  e1.chg_dt
                INNER JOIN ddw01.empl_reltn AS e2
                  ON e2.emp_id = c3.emp_id
                 AND c3.snap_dt >= e2.captr_dt
                 AND c3.snap_dt <  e2.chg_dt
            ) AS emp
              ON emp.evnt_id = c360.evnt_id
            WHERE c360.evnt_id IS NOT NULL
              AND c360.evnt_dt BETWEEN DATE '{wk_start_lit}' AND DATE '{wk_end_lit}'
        """
        with conn.cursor() as cur2:
            cur2.execute(sql_detail)
            rows2 = cur2.fetchall()
            cols2 = [d[0] for d in cur2.description]
            c360_detail_pre = pd.DataFrame(rows2, columns=cols2)

        conn.close()
    except Exception as e:
        logging.error("query failed: %s", e)

# Save outputs to runday folder
c360_short.to_csv(runday_folder / "c360_short.csv", index=False)
c360_detail_pre.to_csv(runday_folder / "c360_detail_pre.csv", index=False)

print(f"\nc360_short rows: {len(c360_short)}  | c360_detail_pre rows: {len(c360_detail_pre)}")


c360_short rows: 0  | c360_detail_pre rows: 0


Screenshot 6 — “Join tool flag, quick frequency, and stage mapping”

What it does (in short):

Left joins tracking_tool_use into c360_detail_pre by OPPOR_ID.

Builds a clean label TOOL_USED = 'Tool Used' when there’s a match, else 'Tool Not Used'.

Prints a PROC FREQ–style frequency table of lob (if that column exists), and saves it.

Creates a Python dict version of the SAS proc format stage mapping (e.g., "Discovery/Understand Needs" → "12.Discovery/Understand Needs"). This is ready to apply once we know the exact stage column name.

In [22]:
# Guards for upstream DataFrames
if 'c360_detail_pre' not in globals():
    c360_detail_pre = pd.DataFrame()
if 'tracking_tool_use' not in globals():
    tracking_tool_use = pd.DataFrame(columns=["OPPOR_ID", "tool_used"])  # empty fallback

c360_detail = c360_detail_pre.copy()

# Align join key if needed
if not c360_detail.empty:
    if 'OPPOR_ID' not in c360_detail.columns and 'rbc_oppor_own_id' in c360_detail.columns:
        c360_detail = c360_detail.rename(columns={"rbc_oppor_own_id": "OPPOR_ID"})

# Join with tool use info
if not c360_detail.empty and 'OPPOR_ID' in c360_detail.columns:
    c360_detail = c360_detail.merge(tracking_tool_use, how='left', on='OPPOR_ID')
else:
    logging.info("OPPOR_ID not found in c360_detail_pre; skipping merge and setting tool_used to None.")
    c360_detail['tool_used'] = None

# TOOL_USED label
c360_detail['TOOL_USED'] = c360_detail['tool_used'].apply(
    lambda v: 'Tool Used' if pd.notnull(v) and str(v).strip() != '' else 'Tool Not Used'
)
if 'tool_used' in c360_detail.columns:
    c360_detail = c360_detail.drop(columns=['tool_used'])

# Stage mapping
stagefmt_map = {
    'Démarche exploratoire/Comprendre le besoin': '11.Démarche exploratoire/Comprendre le besoin',
    'Discovery/Understand Needs': '12.Discovery/Understand Needs',
    'Review Options': '21.Review Options',
    'Present/Gain Commitment': '31.Present/Gain Commitment',
    'Intégration commencée': '41.Intégration commencée',
    'Onboarding Started': '42.Onboarding Started',
    'Opportunity Lost': '51.Opportunity Lost',
    'Opportunity Won': '61.Opportunity Won',
}
if 'stage' in c360_detail.columns:
    c360_detail['stage_fmt'] = c360_detail['stage'].map(stagefmt_map).fillna(c360_detail['stage'])

# Save
c360_detail.to_csv(runday_folder / 'c360_detail.csv', index=False)
print(f"\nc360_detail rows: {len(c360_detail)}")

# PROC FREQ equivalent for `lob`
if 'lob' in c360_detail.columns:
    lob_freq = c360_detail['lob'].value_counts(dropna=False).reset_index()
    lob_freq.columns = ['lob', 'count']
    print("\n[Screenshot 6] lob frequency:\n", lob_freq.to_string(index=False))
    lob_freq.to_csv(runday_folder / 'c360_detail_lob_freq.csv', index=False)
else:
    logging.info("Column 'lob' not found in c360_detail; skipping frequency table.")


c360_detail rows: 0


In [23]:
# =============================================================================
# Screenshot 7 — AOT pull (14‑day lookback) + link flag on c360
# What it does: Pulls AOT events grouped by OPPOR_ID for the window
# [wk_start_min14 .. wk_end], makes a distinct OPPOR_ID list, then LEFT JOINs
# into `c360_detail` and creates `C360_PDA_Link_AOT` = 1 when
#   PROD_CATG_NM == 'Personal Accounts' AND OPPOR_ID is present in AOT.
# Also adds a formatted stage name if `oppor_stage_nm` exists via `stagefmt_map`.
# Saves outputs as CSV.
# =============================================================================

In [24]:
wk_start_min14_lit = wk_start_min14.strftime("%Y-%m-%d")
wk_end_lit = wk_end.strftime("%Y-%m-%d")

aot_all_oppor = pd.DataFrame(columns=["OPPOR_ID", "count_aot"])  # default empty
conn = get_teradata_conn()
if conn is not None:
    try:
        sql_aot = f"""
            SELECT OPPOR_ID,
                   COUNT(*) AS count_aot
            FROM ddw01.evnt_prod_aot
            WHERE ess_src_evnt_dt BETWEEN DATE '{wk_start_min14_lit}' AND DATE '{wk_end_lit}'
              AND OPPOR_ID IS NOT NULL
            GROUP BY 1
        """
        with conn.cursor() as cur:
            cur.execute(sql_aot)
            rows = cur.fetchall()
            cols = [d[0] for d in cur.description]
            aot_all_oppor = pd.DataFrame(rows, columns=cols)
        conn.close()
    except Exception as e:
        logging.error("AOT query failed: %s", e)

# Unique OPPOR_ID list
aot_all_oppor_unique = aot_all_oppor[["OPPOR_ID"]].drop_duplicates().copy()

# Ensure we have c360_detail from prior step
if 'c360_detail' not in globals():
    c360_detail = pd.DataFrame()

# LEFT JOIN on OPPOR_ID
c360_detail_link_aot = c360_detail.copy()
if not c360_detail_link_aot.empty and 'OPPOR_ID' in c360_detail_link_aot.columns:
    c360_detail_link_aot = c360_detail_link_aot.merge(
        aot_all_oppor_unique.rename(columns={"OPPOR_ID": "aot_oppor_id"}),
        how="left",
        left_on="OPPOR_ID",
        right_on="aot_oppor_id",
    )
else:
    logging.info("OPPOR_ID not found in c360_detail; skipping AOT join.")
    c360_detail_link_aot['aot_oppor_id'] = None

# Add link flag
if 'PROD_CATG_NM' not in c360_detail_link_aot.columns:
    c360_detail_link_aot['PROD_CATG_NM'] = None

c360_detail_link_aot['C360_PDA_Link_AOT'] = (
    (c360_detail_link_aot['PROD_CATG_NM'] == 'Personal Accounts') &
    c360_detail_link_aot['aot_oppor_id'].notna()
).astype(int)

# Apply stage mapping if raw column exists
if 'oppor_stage_nm' in c360_detail_link_aot.columns:
    c360_detail_link_aot['oppor_stage_nm_f'] = (
        c360_detail_link_aot['oppor_stage_nm'].map(stagefmt_map)
        .fillna(c360_detail_link_aot['oppor_stage_nm'])
    )

# Save CSV outputs
aot_all_oppor.to_csv(runday_folder / 'aot_all_oppor.csv', index=False)
aot_all_oppor_unique.to_csv(runday_folder / 'aot_all_oppor_unique.csv', index=False)
c360_detail_link_aot.to_csv(runday_folder / 'c360_detail_link_aot.csv', index=False)

print(
    f"\nAOT rows: {len(aot_all_oppor)} | "
    f"Unique OPPOR_IDs: {len(aot_all_oppor_unique)} | "
    f"c360_detail_link_aot rows: {len(c360_detail_link_aot)}"
)


AOT rows: 0 | Unique OPPOR_IDs: 0 | c360_detail_link_aot rows: 0


In [25]:
# =============================================================================
# Screenshot 8 — Filter to PRE cohort + validate PA rationale text
# What it does:
#  1) From `c360_detail_link_aot`, creates two datasets:
#     - `c360_detail_more`: a direct copy (for downstream use).
#     - `c360_detail_more_in_pre`: subset where all hold:
#          asct_prod_fml_nm != 'Risk Protection',
#          lob == 'Retail',
#          C360_PDA_Link_AOT == 1,
#          oppor_stage_nm in ('Opportunity Won','Opportunity Lost').
#  2) Builds `pa_rationale` from the PRE subset for rows with
#     IS_PROD_APRP_FOR_CLNT == 'Not Appropriate - Rationale'.
#     It normalizes rationale text and flags it as VALID when:
#        (a) length > 5 chars,
#        (b) not just a single repeated character,
#        (c) has at least 2 alphanumeric characters.
# Saves all outputs as CSVs.
# =============================================================================

In [30]:
# Guard for upstream frame
if 'c360_detail_link_aot' not in globals():
    c360_detail_link_aot = pd.DataFrame()

c360_detail_more = c360_detail_link_aot.copy()

# PRE cohort mask (safe even if some columns are missing)
mask = pd.Series(True, index=c360_detail_more.index)
if 'asct_prod_fml_nm' in c360_detail_more.columns:
    mask &= c360_detail_more['asct_prod_fml_nm'].ne('Risk Protection')
if 'lob' in c360_detail_more.columns:
    mask &= c360_detail_more['lob'].eq('Retail')
if 'C360_PDA_Link_AOT' in c360_detail_more.columns:
    mask &= c360_detail_more['C360_PDA_Link_AOT'].fillna(0).astype(int).eq(1)
if 'oppor_stage_nm' in c360_detail_more.columns:
    mask &= c360_detail_more['oppor_stage_nm'].isin(['Opportunity Won', 'Opportunity Lost'])

c360_detail_more_in_pre = c360_detail_more[mask].copy()

# Save both cohorts
c360_detail_more.to_csv(runday_folder / 'c360_detail_more.csv', index=False)
c360_detail_more_in_pre.to_csv(runday_folder / 'c360_detail_more_in_pre.csv', index=False)

print(f"PRE cohort rows: {len(c360_detail_more_in_pre)} / {len(c360_detail_more)} total")

# ---- PA rationale extraction & validation ----

def normalize_rationale(txt):
    if pd.isna(txt):
        return ''
    # collapse whitespace, trim, uppercase (similar to SAS compress/translate/strip + upcase)
    return ' '.join(str(txt).split()).upper()

# Select rows that contain PA rationale
if not c360_detail_more_in_pre.empty and 'IS_PROD_APRP_FOR_CLNT' in c360_detail_more_in_pre.columns:
    pa_mask = c360_detail_more_in_pre['IS_PROD_APRP_FOR_CLNT'].eq('Not Appropriate - Rationale')
    pa_rationale = c360_detail_more_in_pre.loc[pa_mask, ['evnt_id', 'IS_PROD_APRP_FOR_CLNT', 'CLNT_RTNL_TXT']].copy()
else:
    pa_rationale = pd.DataFrame(columns=['evnt_id','IS_PROD_APRP_FOR_CLNT','CLNT_RTNL_TXT'])

if not pa_rationale.empty:
    pa_rationale['rationale_clean'] = pa_rationale['CLNT_RTNL_TXT'].apply(normalize_rationale)

    def is_valid_rationale(s):
        s = s or ''
        # (a) length > 5
        if len(s) <= 5:
            return False
        # remove spaces for pattern checks
        s_no_sp = s.replace(' ', '')
        # (b) not only one character repeated
        if s_no_sp and len(set(s_no_sp)) == 1:
            return False
        # (c) at least 2 alphanumeric characters
        if sum(ch.isalnum() for ch in s_no_sp) < 2:
            return False
        return True

    pa_rationale['is_valid_rationale'] = pa_rationale['rationale_clean'].apply(is_valid_rationale)
    pa_rationale['prod_not_appr_rtnl_txt_cat'] = pa_rationale['is_valid_rationale'].map({True: 'VALID', False: 'INVALID'})

# Save
pa_rationale.to_csv(runday_folder / 'pa_rationale.csv', index=False)
print(f"PA rationale rows: {len(pa_rationale)} (valid: {int(pa_rationale.get('is_valid_rationale', pd.Series(dtype=bool)).sum()) if 'is_valid_rationale' in pa_rationale.columns else 0})")


PRE cohort rows: 0 / 0 total
PA rationale rows: 0 (valid: 0)


In [31]:
# =============================================================================
# Screenshot 9 — Merge rationale back + final text category
# What it does:
#  - LEFT JOINs `pa_rationale` (VALID/INVALID) onto the PRE cohort rows
#    (`c360_detail_more_in_pre`) by `evnt_id`.
#  - Builds a single label `prod_not_appr_rtnl_txt_cat` with simple rules:
#       * missing IS_PROD_APRP_FOR_CLNT  → 'Not Available'
#       * IS_PROD_APRP_FOR_CLNT != 'Not Appropriate - Rationale' → 'Not Applicable'
#       * otherwise (it equals 'Not Appropriate - Rationale') → use VALID/INVALID
#         from `pa_rationale` (defaults to 'INVALID' if missing).
#  - Saves `c360_detail_more_in.csv`.
# =============================================================================

In [35]:
# Guards
if 'c360_detail_more_in_pre' not in globals():
    c360_detail_more_in_pre = pd.DataFrame()
if 'pa_rationale' not in globals():
    pa_rationale = pd.DataFrame(columns=['evnt_id','prod_not_appr_rtnl_txt_cat'])

# Prepare merge input
if not pa_rationale.empty and 'evnt_id' in pa_rationale.columns:
    _to_merge = pa_rationale[['evnt_id','prod_not_appr_rtnl_txt_cat']].copy()
else:
    _to_merge = pd.DataFrame(columns=['evnt_id','prod_not_appr_rtnl_txt_cat'])

# Merge if evnt_id exists in left side, otherwise just copy
if 'evnt_id' in c360_detail_more_in_pre.columns:
    c360_detail_more_in = c360_detail_more_in_pre.merge(_to_merge, how='left', on='evnt_id')
else:
    logging.info("evnt_id not found in c360_detail_more_in_pre; skipping merge.")
    c360_detail_more_in = c360_detail_more_in_pre.copy()
    c360_detail_more_in['prod_not_appr_rtnl_txt_cat'] = None  # placeholder

# Final label assignment
def final_rationale_label(row):
    v = row['IS_PROD_APRP_FOR_CLNT'] if 'IS_PROD_APRP_FOR_CLNT' in row else None
    if pd.isna(v) or str(v).strip() == '':
        return 'Not Available'
    if str(v).strip().upper() != 'NOT APPROPRIATE - RATIONALE':
        return 'Not Applicable'
    # if it is 'Not Appropriate - Rationale', use the VALID/INVALID value
    cat = row.get('prod_not_appr_rtnl_txt_cat')
    return cat if isinstance(cat, str) and cat else 'INVALID'

c360_detail_more_in['prod_not_appr_rtnl_txt_cat'] = c360_detail_more_in.apply(final_rationale_label, axis=1)

# Save
c360_detail_more_in.to_csv(runday_folder / 'c360_detail_more_in.csv', index=False)
print(f"c360_detail_more_in rows: {len(c360_detail_more_in)}")

# PROC FREQ equivalent
if 'prod_not_appr_rtnl_txt_cat' in c360_detail_more_in.columns:
    freq = c360_detail_more_in['prod_not_appr_rtnl_txt_cat'].value_counts(dropna=False).reset_index()
    freq.columns = ['prod_not_appr_rtnl_txt_cat','count']
    print("\nRationale category frequency:\n", freq.to_string(index=False))
    freq.to_csv(runday_folder / 'c360_detail_more_in_rationale_freq.csv', index=False)

c360_detail_more_in rows: 0

Rationale category frequency:
 Empty DataFrame
Columns: [prod_not_appr_rtnl_txt_cat, count]
Index: []


Screenshot 10 — “Comment map + per-opportunity counter”

What it does (in short):

Adds a Python dict that mirrors the SAS $cs_cmt format mapping (COM1…COM19 → readable comments like “Test population (less samples)”).

Sorts c360_detail_more_in by OPPOR_ID and creates level_oppor = row number within each opportunity (1,2,3,…) — equivalent to the SAS first.OPPOR_ID + level_oppor+1 pattern.

Saves c360_detail_more_in_level.csv.

In [38]:
# Comment/CS mapping (SAS $cs_cmt → Python dict)
cs_cmt_map = {
    'COM1':  'Test population (less samples)',
    'COM2':  'Match population',
    'COM3':  'Mismatch population (less samples)',
    'COM4':  'Non Anomaly Population',
    'COM5':  'Anomaly Population',
    'COM6':  'Number of Deposit Sessions',
    'COM7':  'Number of Accounts',
    'COM8':  'Number of Transactions',
    'COM9':  'Non Blank Population',
    'COM10': 'Blank Population',
    'COM11': 'Unable to Assess',
    'COM12': 'Number of Failed Data Elements',
    'COM13': 'Population Distribution',
    'COM14': 'Reconciled Population',
    'COM15': 'Not Reconciled Population',
    'COM16': 'Pass',
    'COM17': 'Fail',
    'COM18': 'Not Applicable',
    'COM19': 'Potential Fail',
}

# Guard
if 'c360_detail_more_in' not in globals():
    c360_detail_more_in = pd.DataFrame()

c360_detail_more_in_leveled = c360_detail_more_in.copy()

if not c360_detail_more_in_leveled.empty:
    # Apply mapping if column exists
    if 'comment_code' in c360_detail_more_in_leveled.columns:
        c360_detail_more_in_leveled['comment_text'] = (
            c360_detail_more_in_leveled['comment_code'].map(cs_cmt_map)
        )

    # Add row-level counter per OPPOR_ID
    if 'OPPOR_ID' in c360_detail_more_in_leveled.columns:
        c360_detail_more_in_leveled = (
            c360_detail_more_in_leveled.sort_values(['OPPOR_ID'])
            .assign(level_oppor=lambda d: d.groupby('OPPOR_ID').cumcount() + 1)
        )
    else:
        logging.info("Column 'OPPOR_ID' not found, skipping level_oppor generation.")

# Save
c360_detail_more_in_leveled.to_csv(runday_folder / 'c360_detail_more_in_level.csv', index=False)
print(f"\nLeveled rows: {len(c360_detail_more_in_leveled)}")


Leveled rows: 0


In [39]:
# =============================================================================
# Screenshot 11 — Final shaped view for audit/export
# What it does:
#  - Takes the first row per opportunity (where level_oppor == 1).
#  - Adds friendly report fields and segments (constants + sourced columns).
#  - Derives date fields: `segment10` as YYYYMM from evnt_dt, `SnapDate` as end
#    of week of evnt_dt, and `DateCompleted` as today.
#  - Builds comment fields from `CommentCode` using `cs_cmt_map`.
#  - Saves the shaped table and a PROC FREQ–style summary of `PROD_CATG_NM`.
# =============================================================================

In [41]:
# Input: leveled cohort from Above (sS-10)
src = 'c360_detail_more_in_leveled' if 'c360_detail_more_in_leveled' in globals() else (
      'c360_detail_more_in' if 'c360_detail_more_in' in globals() else None)

if src is None:
    tmp_pa_c360_4ac = pd.DataFrame()
else:
    df0 = globals()[src].copy()

    # filter only first rows per OPPOR_ID if column exists
    if 'level_oppor' in df0.columns:
        df = df0[df0['level_oppor'].eq(1)].copy()
    else:
        df = df0.copy()

    # Safe datetime parse for evnt_dt
    if 'evnt_dt' in df.columns:
        evnt_dt_parsed = pd.to_datetime(df['evnt_dt'], errors='coerce')
    else:
        evnt_dt_parsed = pd.to_datetime(pd.Series([], dtype='datetime64[ns]'))

    # Constants
    df['RegulatoryName'] = 'C86'
    df['LOB'] = 'Retail'  # constant per the SAS snippet
    df['ReportName'] = 'C86 Client360 Product Appropriateness'
    df['ControlRisk'] = 'Completeness'
    df['TestType'] = 'Anomaly'
    df['TestPeriod'] = 'Origination'

    # ProductType → Product Category (try multiple candidates)
    prod_cat_col = None
    for c in ['ASCT_PROD_FMLY_NM', 'asct_prod_fml_nm', 'asct_prod_fmly_nm', 'PROD_CATG_NM']:
        if c in df.columns:
            prod_cat_col = c
            break
    df['ProductType'] = df[prod_cat_col] if prod_cat_col else ''

    # Segments (safe guards for missing cols)
    df['segment1'] = 'Account Open'
    df['segment2'] = df['ProductType']
    df['segment3'] = df['PROD_SRVC_NM'] if 'PROD_SRVC_NM' in df.columns else ''
    df['segment6'] = df['oppor_stage_nm'] if 'oppor_stage_nm' in df.columns else ''
    df['segment7'] = df['TOOL_USED'] if 'TOOL_USED' in df.columns else ''

    # segment10 = YYYYMM from event date
    df['segment10'] = evnt_dt_parsed.dt.strftime('%Y%m') if len(evnt_dt_parsed) else ''

    # Comments
    df['CommentCode'] = 'COM13'
    df['Comments'] = df['CommentCode'].map(cs_cmt_map)

    # Holdout flag
    df['HoldoutFlag'] = 'N'

    # Dates
    if len(evnt_dt_parsed):
        dow = evnt_dt_parsed.dt.weekday  # Monday=0..Sunday=6
        df['SnapDate'] = (evnt_dt_parsed + pd.to_timedelta(6 - dow, unit='D')).dt.strftime('%Y-%m-%d')
    else:
        df['SnapDate'] = ''
    df['DateCompleted'] = pd.Timestamp.today().strftime('%Y-%m-%d')

    # Final shaped view
    keep_cols = [
        'OPPOR_ID', 'evnt_id', 'evnt_dt', 'LOB', 'RegulatoryName', 'ReportName',
        'ControlRisk', 'TestType', 'TestPeriod', 'ProductType',
        'segment1','segment2','segment3','segment6','segment7','segment10',
        'CommentCode','Comments','HoldoutFlag','SnapDate','DateCompleted'
    ]
    keep_cols = [c for c in keep_cols if c in df.columns]
    tmp_pa_c360_4ac = df[keep_cols].copy()

# Save shaped table
tmp_pa_c360_4ac.to_csv(runday_folder / 'tmp_pa_c360_4ac.csv', index=False)
print(f"tmp_pa_c360_4ac rows: {len(tmp_pa_c360_4ac)}")

# PROC FREQ–style summary for PROD_CATG_NM
if src is not None and 'PROD_CATG_NM' in globals()[src].columns:
    freq_prod = globals()[src]['PROD_CATG_NM'].value_counts(dropna=False).reset_index()
    freq_prod.columns = ['PROD_CATG_NM','count']
    freq_prod.to_csv(runday_folder / 'prod_catg_freq.csv', index=False)
    print("PROD_CATG_NM frequency saved.")

tmp_pa_c360_4ac rows: 0
PROD_CATG_NM frequency saved.


Screenshot 12 — “AC assessment roll-up (segment labels + counts)”

What it does (in short):

Starts from tmp_pa_c360_4ac and left joins IS_PROD_APRP_FOR_CLNT and prod_not_appr_rtnl_txt_cat (from c360_detail_more_in) by evnt_id.

Adds RDE='PA002_Client360_Completeness_RDE'.

Builds segment4 from IS_PROD_APRP_FOR_CLNT:

outside C360 → Product Appropriateness assessed outside Client 360

Not Appropriate – Rationale → Product Not Appropriate

Client declined… → Client declined product appropriateness assessment

Product Appropriate → Product Appropriate

else → Missing

Sets segment5 = prod_not_appr_rtnl_txt_cat.

Groups by the report/segment fields and outputs counts (volume and a simple Amount = volume).

Saves: tmp_pa_c360_ac_assessment.csv and a PROC FREQ–style segment4_freq.csv.

In [43]:
# Ensure required DataFrames exist
if 'tmp_pa_c360_4ac' not in globals():
    tmp_pa_c360_4ac = pd.DataFrame()
if 'c360_detail_more_in' not in globals():
    c360_detail_more_in = pd.DataFrame()

# Start from a copy of the base DataFrame
base = tmp_pa_c360_4ac.copy()

# Bring in needed columns from the detailed cohort
merge_cols = ['evnt_id', 'IS_PROD_APRP_FOR_CLNT', 'prod_not_appr_rtnl_txt_cat']
right = c360_detail_more_in[[c for c in merge_cols if c in c360_detail_more_in.columns]].copy()

if not right.empty:
    base = base.merge(right, how='left', on='evnt_id')
else:
    # Ensure columns exist even if right is empty
    for c in merge_cols[1:]:
        if c not in base.columns:
            base[c] = None

# Constants and segment derivations
base['RDE'] = 'PA002_Client360_Completeness_RDE'

# Map segment4 values
map_seg4 = {
    'Product Appropriateness assessed outside Client 360': 'Product Appropriateness assessed outside Client 360',
    'Not Appropriate - Rationale': 'Product Not Appropriate',
    'Client declined product appropriateness assessment': 'Client declined product appropriateness assessment',
    'Product Appropriate': 'Product Appropriate',
}
base['segment4'] = base['IS_PROD_APRP_FOR_CLNT'].map(map_seg4).fillna('Missing')

# segment5 is the rationale text category
base['segment5'] = base.get('prod_not_appr_rtnl_txt_cat', None)

# Columns to group by (only include those present)
group_cols_all = [
    'RegulatoryName','LOB','ReportName','ControlRisk','TestType','TestPeriod','ProductType',
    'RDE','segment1','segment2','segment3','segment4','segment5','segment6','segment7',
    'segment10','HoldoutFlag','CommentCode','Comments','DateCompleted','SnapDate'
]
group_cols = [c for c in group_cols_all if c in base.columns]

# Aggregate/roll-up
if group_cols:
    rollup = base.groupby(group_cols, dropna=False).size().reset_index(name='volume')
    rollup['Amount'] = rollup['volume']  # beginner-friendly roll-up
    tmp_pa_c360_ac_assessment = rollup
else:
    tmp_pa_c360_ac_assessment = pd.DataFrame()

# Save outputs
tmp_pa_c360_ac_assessment.to_csv(runday_folder / 'tmp_pa_c360_ac_assessment.csv', index=False)
print(f"assessment rows: {len(tmp_pa_c360_ac_assessment)}")

# PROC FREQ-style count of segment4
if 'segment4' in base.columns:
    seg4_freq = base['segment4'].value_counts(dropna=False).reset_index()
    seg4_freq.columns = ['segment4','count']
    seg4_freq.to_csv(runday_folder / 'segment4_freq.csv', index=False)
    print("segment4 frequency saved.")

assessment rows: 0
segment4 frequency saved.


In [44]:
# =============================================================================
# Screenshot 13 — Output template + aligned export
# What it does:
#  - Defines a fixed column order/types (like SAS template dataset).
#  - Takes `tmp_pa_c360_ac_assessment` and reindexes/renames to match the
#    template, filling missing columns with blanks/zeros.
#  - Formats dates (`DateCompleted`, `SnapDate`) as YYYY-MM-DD (similar to yymmdd10.).
#  - Saves final export `pa_c360_autocomplete_tool_use.csv`.
# =============================================================================


In [45]:
# Guard
if 'tmp_pa_c360_ac_assessment' not in globals():
    tmp_pa_c360_ac_assessment = pd.DataFrame()

# Desired column order (Python version of the SAS template)
TEMPLATE_COLS = [
    'RegulatoryName','LOB','ReportName','ControlRisk','TestType','TestPeriod','ProductType',
    'RDE','segment','segment2','segment3','segment4','segment5','segment6','segment7','segment8','segment9','segment10',
    'HoldoutFlag','CommentCode','Comments','DateCompleted','SnapDate','volume','Amount'
]

# Start from the assessment roll-up
export_df = tmp_pa_c360_ac_assessment.copy()

# In our earlier steps we used 'volume' (lowercase) already; make sure it's there
if 'volume' not in export_df.columns and 'Volume' in export_df.columns:
    export_df.rename(columns={'Volume':'volume'}, inplace=True)

# Map segment1 → segment if needed
if 'segment' not in export_df.columns and 'segment1' in export_df.columns:
    export_df['segment'] = export_df['segment1']

# Ensure all template columns exist
for c in TEMPLATE_COLS:
    if c not in export_df.columns:
        export_df[c] = '' if c not in ('volume','Amount') else 0

# Coerce simple types
export_df['volume'] = pd.to_numeric(export_df['volume'], errors='coerce').fillna(0).astype(int)
export_df['Amount'] = pd.to_numeric(export_df['Amount'], errors='coerce').fillna(export_df['volume']).astype(int)

# Date formatting (YYYY-MM-DD ~ yymmdd10.)
for dc in ['DateCompleted','SnapDate']:
    if dc in export_df.columns:
        export_df[dc] = pd.to_datetime(export_df[dc], errors='coerce').dt.strftime('%Y-%m-%d')

# Final order
export_df = export_df[TEMPLATE_COLS]

# Save
export_df.to_csv(runday_folder / 'pa_c360_autocomplete_tool_use.csv', index=False)
print(f"\nFinal export rows: {len(export_df)}")


Final export rows: 0


In [47]:
# =============================================================================
# Screenshot 14 — Prepare data for 'tool used' count (add segment8 = tool name)
# =============================================================================

# Ensure required DataFrames exist
if 'tmp_pa_c360_4ac' not in globals():
    tmp_pa_c360_4ac = pd.DataFrame()
if 'tracking_tool_use_distinct' not in globals():
    tracking_tool_use_distinct = pd.DataFrame(columns=['OPPOR_ID','ADVC_TOOL_NM'])

# Copy base DataFrames
left_df = tmp_pa_c360_4ac.copy()
right_df = tracking_tool_use_distinct.copy()

# Ensure OPPOR_ID exists
for df in [left_df, right_df]:
    if 'OPPOR_ID' not in df.columns:
        df['OPPOR_ID'] = None

# Merge and create segment8
count_pre = left_df.merge(
    right_df[['OPPOR_ID','ADVC_TOOL_NM']].drop_duplicates(),
    how='left',
    on='OPPOR_ID'
)

count_pre['segment8'] = count_pre.get('ADVC_TOOL_NM', '').astype(str).str.upper()

# Fill in default values for mandatory columns
defaults = {
    'RegulatoryName': 'C86',
    'LOB': 'Retail',
    'ReportName': 'C86 Client360 Product Appropriateness',
    'ControlRisk': 'Completeness',
    'TestType': 'Anomaly',
    'TestPeriod': 'Origination'
}
for col, default in defaults.items():
    count_pre[col] = count_pre.get(col, default)

# SnapDate calculation (end of week)
if 'SnapDate' not in count_pre.columns and 'evnt_dt' in count_pre.columns:
    evnt_dt_parsed = pd.to_datetime(count_pre['evnt_dt'], errors='coerce')
    dow = evnt_dt_parsed.dt.weekday
    count_pre['SnapDate'] = (evnt_dt_parsed + pd.to_timedelta(6 - dow, unit='D')).dt.strftime('%Y-%m-%d')

# DateCompleted default to today
if 'DateCompleted' not in count_pre.columns:
    count_pre['DateCompleted'] = pd.Timestamp.today().strftime('%Y-%m-%d')

# Save output
count_pre.to_csv(runday_folder / 'tmp_pa_c360_4ac_count.csv', index=False)
print(f"\n tmp_pa_c360_4ac_count rows: {len(count_pre)}")


 tmp_pa_c360_4ac_count rows: 0


In [49]:
# Ensure c360_detail_more_in exists
if 'c360_detail_more_in' not in globals():
    c360_detail_more_in = pd.DataFrame()

# Start from a copy of the tool-used dataset
base2 = count_pre.copy()

# Columns to bring in from the detailed cohort
right2_cols = ['evnt_id','IS_PROD_APRP_FOR_CLNT','prod_not_appr_rtnl_txt_cat']
right2 = c360_detail_more_in[[c for c in right2_cols if c in c360_detail_more_in.columns]].copy()

# Merge or create missing columns
if not right2.empty:
    base2 = base2.merge(right2, how='left', on='evnt_id')
else:
    for c in right2_cols[1:]:
        if c not in base2.columns:
            base2[c] = None

# Constants and segment derivations
base2['RDE'] = 'PA003_Client360_Completeness_Tool'

# Map segment4 values using existing map_seg4
base2['segment4'] = base2.get('IS_PROD_APRP_FOR_CLNT', None).map(map_seg4).fillna('Missing')

# segment5 is the rationale text category
base2['segment5'] = base2.get('prod_not_appr_rtnl_txt_cat', None)

# Columns to group by (only include those present)
count_group_cols_all = [
    'RegulatoryName','LOB','ReportName','ControlRisk','TestType','TestPeriod','ProductType',
    'RDE','segment1','segment2','segment3','segment4','segment5','segment6','segment7',
    'segment8','segment10','HoldoutFlag','CommentCode','Comments','DateCompleted','SnapDate'
]
count_group_cols = [c for c in count_group_cols_all if c in base2.columns]

# Aggregate / roll-up
if count_group_cols:
    count_rollup = base2.groupby(count_group_cols, dropna=False).size().reset_index(name='volume')
    count_rollup['Amount'] = count_rollup['volume']  # beginner-friendly roll-up
    tmp_pa_c360_ac_count_assessment = count_rollup
else:
    tmp_pa_c360_ac_count_assessment = pd.DataFrame()

# Save output
tmp_pa_c360_ac_count_assessment.to_csv(runday_folder / 'tmp_pa_c360_ac_count_assessment.csv', index=False)
print(f"ac_count_assessment rows: {len(tmp_pa_c360_ac_count_assessment)}")

ac_count_assessment rows: 0


In [52]:
# -------------------------------
# Screenshot 16 — Align tool-count assessment to template and combine
# -------------------------------

# Ensure tmp_pa_c360_ac_count_assessment exists
if 'tmp_pa_c360_ac_count_assessment' not in globals():
    tmp_pa_c360_ac_count_assessment = pd.DataFrame()

# Segment4 frequency for tool-count assessment
if 'segment4' in tmp_pa_c360_ac_count_assessment.columns:
    seg4_count_freq = tmp_pa_c360_ac_count_assessment['segment4'].value_counts(dropna=False).reset_index()
    seg4_count_freq.columns = ['segment4', 'count']
    seg4_count_freq.to_csv(runday_folder / 'segment4_freq_count.csv', index=False)

# Template columns
if 'TEMPLATE_COLS' not in globals():
    TEMPLATE_COLS = [
        'RegulatoryName','LOB','ReportName','ControlRisk','TestType','TestPeriod','ProductType',
        'RDE','segment','segment2','segment3','segment4','segment5','segment6','segment7','segment8','segment9','segment10',
        'HoldoutFlag','CommentCode','Comments','DateCompleted','SnapDate','volume','Amount'
    ]

# Align tmp_pa_c360_ac_count_assessment to template
pa_c360_autocomplete_Count_Tool = tmp_pa_c360_ac_count_assessment.copy()
if 'segment' not in pa_c360_autocomplete_Count_Tool.columns and 'segment1' in pa_c360_autocomplete_Count_Tool.columns:
    pa_c360_autocomplete_Count_Tool['segment'] = pa_c360_autocomplete_Count_Tool['segment1']

# Ensure all template columns exist
for c in TEMPLATE_COLS:
    if c not in pa_c360_autocomplete_Count_Tool.columns:
        pa_c360_autocomplete_Count_Tool[c] = '' if c not in ('volume','Amount') else 0

# Ensure numeric columns are correct
pa_c360_autocomplete_Count_Tool['volume'] = pd.to_numeric(pa_c360_autocomplete_Count_Tool['volume'], errors='coerce').fillna(0).astype(int)
pa_c360_autocomplete_Count_Tool['Amount'] = pd.to_numeric(pa_c360_autocomplete_Count_Tool['Amount'], errors='coerce')\
                                            .fillna(pa_c360_autocomplete_Count_Tool['volume']).astype(int)

# Standardize date columns
for dc in ['DateCompleted','SnapDate']:
    if dc in pa_c360_autocomplete_Count_Tool.columns:
        pa_c360_autocomplete_Count_Tool[dc] = pd.to_datetime(pa_c360_autocomplete_Count_Tool[dc], errors='coerce')\
                                              .dt.strftime('%Y-%m-%d')

# Reorder to template
pa_c360_autocomplete_Count_Tool = pa_c360_autocomplete_Count_Tool[TEMPLATE_COLS]

# Load tool-use export
if 'export_df' in globals():
    pa_c360_autocomplete_tool_use = export_df.copy()
else:
    tool_use_path = runday_folder / 'pa_c360_autocomplete_tool_use.csv'
    if tool_use_path.exists():
        pa_c360_autocomplete_tool_use = pd.read_csv(tool_use_path)
    else:
        pa_c360_autocomplete_tool_use = pd.DataFrame(columns=TEMPLATE_COLS)

# Ensure all template columns exist for export
for c in TEMPLATE_COLS:
    if c not in pa_c360_autocomplete_tool_use.columns:
        pa_c360_autocomplete_tool_use[c] = '' if c not in ('volume','Amount') else 0
pa_c360_autocomplete_tool_use = pa_c360_autocomplete_tool_use[TEMPLATE_COLS]

# Combine count-tool assessment with tool-use export
combine_pa_autocomplete = pd.concat([pa_c360_autocomplete_Count_Tool, pa_c360_autocomplete_tool_use], ignore_index=True)
combine_pa_autocomplete.to_csv(runday_folder / 'combine_pa_autocomplete.csv', index=False)

# Append into base table (replace rows for today's DateCompleted)
base_path = outpath / 'pa_client360_autocomplete.csv'
if base_path.exists():
    base_df = pd.read_csv(base_path)
else:
    base_df = pd.DataFrame(columns=TEMPLATE_COLS)

# Ensure all template columns exist in base
for c in TEMPLATE_COLS:
    if c not in base_df.columns:
        base_df[c] = '' if c not in ('volume','Amount') else 0

# Filter out today's rows and append
runday_date_str = pd.to_datetime(runday, format='%Y%m%d').strftime('%Y-%m-%d')
if 'DateCompleted' in base_df.columns:
    base_df = base_df[base_df['DateCompleted'] != runday_date_str]

final_autocomplete = pd.concat([base_df[TEMPLATE_COLS], combine_pa_autocomplete[TEMPLATE_COLS]], ignore_index=True)
final_autocomplete.to_csv(base_path, index=False)

print(f"combined rows: {len(combine_pa_autocomplete)} | final table rows: {len(final_autocomplete)}")

combined rows: 0 | final table rows: 0


In [54]:
# -------------------------------
# Screenshot 17 — Export autocomplete workbook and build daily detail extract
# -------------------------------

excel_path = outpath / 'pa_client360_autocomplete.xlsx'
base_csv_path = outpath / 'pa_client360_autocomplete.csv'

# Ensure final_autocomplete exists
if 'final_autocomplete' not in globals():
    if base_csv_path.exists():
        final_autocomplete = pd.read_csv(base_csv_path)
    else:
        final_autocomplete = pd.DataFrame()

# Prepare detail build from enriched cohort
detail_src = c360_detail_more_in.copy() if 'c360_detail_more_in' in globals() else pd.DataFrame()

# Merge in tool use if available
if 'tracking_tool_use_distinct' in globals() and not detail_src.empty:
    _tool = tracking_tool_use_distinct[['OPPOR_ID','ADVC_TOOL_NM']].drop_duplicates()
    detail_src = detail_src.merge(_tool, how='left', on='OPPOR_ID')
else:
    if 'ADVC_TOOL_NM' not in detail_src.columns:
        detail_src['ADVC_TOOL_NM'] = ''

# Ensure all columns exist to avoid KeyErrors
for col in ['evnt_dt','EVNT_TMSTP','DateCompleted','OPPOR_ID','OPPOR_REC_TYP','PROD_CD',
            'PROD_CATG_NM','ASCT_PROD_FMLY_NM','PROD_SRVC_NM','oppor_stage_nm','TOOL_USED',
            'tool_used','IS_PROD_APRP_FOR_CLNT','CLNT_RTNL_TXT','prod_not_appr_rtnl_txt_cat',
            'RBC_OPPOR_OWN_ID','OCCPT_JOB_CD','HR_POSN_TITL_EN','ORG_UNT_NO','POSN_STRT_DT']:
    if col not in detail_src.columns:
        detail_src[col] = pd.NA

# Safe datetime conversions
_ev = pd.to_datetime(detail_src['evnt_dt'], errors='coerce')
_event_month = _ev.dt.strftime('%Y%m')
_week_end = (_ev + pd.to_timedelta(6 - _ev.dt.weekday, unit='D')).dt.strftime('%Y-%m-%d')
_event_date = _ev.dt.strftime('%Y-%m-%d')

_ts = pd.to_datetime(detail_src['EVNT_TMSTP'], errors='coerce').fillna(_ev)
_ts = _ts.dt.strftime('%Y-%m-%d %I:%M:%S %p')

# Map PA results
_pa_map = {
    'Product Appropriateness assessed outside Client 360': 'Product Appropriateness assessed outside Client 360',
    'Not Appropriate - Rationale': 'Product Not Appropriate',
    'Client declined product appropriateness assessment': 'Client declined product appropriateness assessment',
    'Product Appropriate': 'Product Appropriate',
}

# Build detail dataframe
detail_df = pd.DataFrame({
    'event_month': _event_month,
    'reporting_date': pd.to_datetime(detail_src['DateCompleted'], errors='coerce').dt.strftime('%Y-%m-%d'),
    'event_week_ending': _week_end,
    'event_date': _event_date,
    'event_timestamp': _ts,
    'opportunity_id': detail_src['OPPOR_ID'],
    'opportunity_type': detail_src['OPPOR_REC_TYP'],
    'product_code': detail_src['PROD_CD'],
    'product_category_name': detail_src['PROD_CATG_NM'],
    'product_family_name': detail_src['ASCT_PROD_FMLY_NM'],
    'product_name': detail_src['PROD_SRVC_NM'],
    'oppor_stage_nm': detail_src['oppor_stage_nm'],
    'tool_used': detail_src['TOOL_USED'].where(detail_src['TOOL_USED'].notna(), detail_src['tool_used']),
    'tool_nm': detail_src['ADVC_TOOL_NM'],
    'PA_result': detail_src['IS_PROD_APRP_FOR_CLNT'].map(_pa_map).fillna('Missing'),
    'PA_rationale': detail_src['CLNT_RTNL_TXT'],
    'PA_rationale_validity': detail_src['prod_not_appr_rtnl_txt_cat'],
    'employee_id': detail_src['RBC_OPPOR_OWN_ID'],
    'job_code': detail_src['OCCPT_JOB_CD'],
    'position_title': detail_src['HR_POSN_TITL_EN'],
    'employee_transit': detail_src['ORG_UNT_NO'],
    'position_start_date': pd.to_datetime(detail_src['POSN_STRT_DT'], errors='coerce').dt.strftime('%Y-%m-%d'),
})

# Save detail CSV
_detail_csv = outpath / f'pa_client360_detail_{runday}.csv'
detail_df.to_csv(_detail_csv, index=False)

# Save Excel workbook with autocomplete and detail
with pd.ExcelWriter(excel_path) as _writer:
    final_autocomplete.to_excel(_writer, index=False, sheet_name='autocomplete')
    detail_df.to_excel(_writer, index=False, sheet_name=f'detail_{runday}')

print(f"detail rows: {len(detail_df)} | Excel: {excel_path.name}")

detail rows: 0 | Excel: pa_client360_autocomplete.xlsx


In [59]:
# -------------------------------
# Screenshot 18 — Filtered detail export and pivot update
# -------------------------------

import os, time

# Filtered PA_result values
filter_vals = [
    'Product Not Appropriate',
    'Missing',
    'Product Appropriateness assessed outside Client 360',
]

# Ensure detail_df exists
if 'detail_df' not in globals():
    detail_df = pd.DataFrame()

# Filter rows safely
if not detail_df.empty:
    filtered_detail = detail_df[detail_df.get('PA_result', pd.Series(dtype=str)).isin(filter_vals)].copy()
else:
    filtered_detail = pd.DataFrame()

# Write filtered detail workbook in run-day folder
_detail_xlsx = runday_folder / f'pa_client360_detail_{runday}.xlsx'
with pd.ExcelWriter(_detail_xlsx) as writer:
    filtered_detail.to_excel(writer, index=False, sheet_name='detail')

# Update pivot workbook with autocomplete sheet
_pivot_xlsx = outpath / 'PA_Client360_Pivot.xlsx'
with pd.ExcelWriter(_pivot_xlsx) as writer:
    final_autocomplete_safe = final_autocomplete if 'final_autocomplete' in globals() else pd.DataFrame()
    final_autocomplete_safe.to_excel(writer, index=False, sheet_name='Autocomplete')

# Change permissions to 777 for files modified in the last 720 minutes (best-effort)
_now = time.time()
for path in outpath.rglob('*'):
    try:
        if path.is_file() and (_now - path.stat().st_mtime) <= 720 * 60:
            os.chmod(path, 0o777)
    except Exception:
        pass

print(f"filtered detail rows: {len(filtered_detail)} | "
      f"detail workbook: {_detail_xlsx.name} | pivot workbook: {_pivot_xlsx.name}")

filtered detail rows: 0 | detail workbook: pa_client360_detail_20250910.xlsx | pivot workbook: PA_Client36e_Pivot.xlsx
